## Image classification

In [1]:
#import libraries
import os
import keras
from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions, preprocess_input
from keras.applications import vgg16
from keras.models import Model
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#model
model = keras.applications.vgg16.VGG16(weights='imagenet', include_top=True)

In [3]:
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [4]:
def load_image(path):
    img = image.load_img(path, target_size=model.input_shape[1:3])
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return img, x


In [5]:
#Model for feature extraction
feat_extractor = Model(inputs=model.input, outputs=model.get_layer("fc2").output)

In [6]:
#recursively crawl the folder specified by image_path looking for images of the extensions inside of image_extensions 
#and then limiting them to a random subset of maximum max_num_images images.

images_path = 'dataset'
image_extensions = ['.jpg', '.png', '.jpeg']   # case-insensitive (upper/lower doesn't matter)
max_num_images = 1000

images = [os.path.join(dp, f) for dp, dn, filenames in os.walk(images_path) for f in filenames if os.path.splitext(f)[1].lower() in image_extensions]
if max_num_images < len(images):
    images = [images[i] for i in sorted(random.sample(xrange(len(images)), max_num_images))]

print("keeping %d images to analyze" % len(images))

keeping 183 images to analyze


In [7]:
#a loop which will open each image, extract its feature vector, and append it to a list called features which will contain our activations for each image
import time
tic = time.perf_counter()


features = []
for i, image_path in enumerate(images):
    if i % 500 == 0:
        toc = time.perf_counter()
        elap = toc-tic;
        tic = time.perf_counter()
#         print("analyzing image %d / %d. Time: %4.4f seconds." % (i, len(images),elap))
    img, x = load_image(image_path);
    feat = feat_extractor.predict(x)[0]
    features.append(feat)

print('finished extracting features for %d images' % len(images))

finished extracting features for 183 images


In [9]:
#Alone, these activations provide a good representation, but it is a good idea to do one more step before using these as our feature vectors, 
#which is to do a principal component analysis (PCA) to reduce the dimensionality of our feature vectors down to 30. We apply PCA for two reasons: 
#1) the 4096-bit feature vector may have some redundancy in it, such that multiple elements in the vector are highly correlated or similar. 
#This would skew similarity comparisons towards those over-represented features. 
#2) Operating over 4096 elements is inefficient both in terms of space/memory requirements and processor speed, and 
#it would be better for us if we can reduce the length of these vectors but maintain the same effective representation. 
#PCA allows us to do this by reducing the dimensionality down of the feature vectors from 4096 to much less, 
#but maintain a representation which is still faithful to the original data, by preserving the relative inter-point distance.
#Thus, PCA reduces the amount of redundancy in our features (from duplicate or highly-correlated features), 
#speeds up computation over them, and reduces the amount of memory they take up.

from sklearn.decomposition import PCA
features = np.array(features)
pca = PCA(n_components=30)
pca.fit(features)

PCA(n_components=30)

In [10]:
pca_features = pca.transform(features)

In [11]:
from scipy.spatial import distance
def get_closest_images(query_image_idx, num_results=10):
    distances = [ distance.cosine(pca_features[query_image_idx], feat) for feat in pca_features ]
    idx_closest = sorted(range(len(distances)), key=lambda k: distances[k])[1:num_results+1]
    return idx_closest

def get_concatenated_images(indexes, thumb_height):
    thumbs = []
    paths = []
    for idx in indexes:
        img = image.load_img(images[idx])
        img = img.resize((int(img.width * thumb_height / img.height), thumb_height))
        paths.append(images[idx])
        thumbs.append(img)
    concat_image = np.concatenate([np.asarray(t) for t in thumbs], axis=1)
    return concat_image,paths



## LOGIN WINDOW

In [25]:
from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtWidgets import QFileDialog,QWidget
from PyQt5.QtGui import QPixmap

class Ui_login_Form(object):
    
    def openSearchwindow(self):
        user_id = self.username_lineEdit.text()
        password = self.password_lineEdit.text()
        
        users = ['Dhanush','Pradyumna','Satwik']
        passwords = ['AIE19002','AIE9043','AIE19059']
        
        if user_id == users[0] and passwords[0] or users[1] and passwords[1] or users[2] and passwords[2]:
            self.window = QtWidgets.QMainWindow()
            self.ui = Ui_SearchWindow()
            self.ui.setupUi(self.window)
            self.window.show()
        else:
            self.caution_label.setText("Invalid Credentials")
        
        
        
    def setupUi(self, Form):
        Form.setObjectName("Form")
        Form.resize(773, 634)
        self.gridLayout = QtWidgets.QGridLayout(Form)
        self.gridLayout.setObjectName("gridLayout")
        self.widget = QtWidgets.QWidget(Form)
        self.widget.setStyleSheet("")
        self.widget.setObjectName("widget")
        
        self.img_label = QtWidgets.QLabel(self.widget)
        self.img_label.setGeometry(QtCore.QRect(0, 0, 401, 611))
        self.img_label.setStyleSheet("border-image: url(shopping.png);\n""border-top-left-radius :50px;")
        self.img_label.setText("")
        self.img_label.setObjectName("img_label")

        self.label_2 = QtWidgets.QLabel(self.widget)
        self.label_2.setGeometry(QtCore.QRect(400, 0, 341, 611))
        self.label_2.setStyleSheet("background-color: rgb(255, 255, 255);\n""border-bottom-right-radius :50px;")
        self.label_2.setText("")
        self.label_2.setObjectName("label_2")

        self.label_3 = QtWidgets.QLabel(self.widget)
        self.label_3.setGeometry(QtCore.QRect(480, 50, 191, 51))
        font = QtGui.QFont()
        font.setPointSize(22)
        font.setBold(True)
        font.setWeight(75)
        self.label_3.setFont(font)
        self.label_3.setAlignment(QtCore.Qt.AlignCenter)
        self.label_3.setObjectName("label_3")

        self.username_lineEdit = QtWidgets.QLineEdit(self.widget)
        self.username_lineEdit.setGeometry(QtCore.QRect(420, 170, 300, 80))
        font = QtGui.QFont()
        font.setPointSize(15)
        self.username_lineEdit.setFont(font)
        self.username_lineEdit.setStyleSheet("background-color: rgb(255, 255, 255);\n""border:none;\n""border-bottom: 3px solid rgba(46,82,101,200);\n""color:rgba(0,0,0,240);\n""padding-bottom:7px;")
        self.username_lineEdit.setObjectName("username_lineEdit")

        self.password_lineEdit = QtWidgets.QLineEdit(self.widget)
        self.password_lineEdit.setGeometry(QtCore.QRect(420, 340, 300, 80))
        font = QtGui.QFont()
        font.setPointSize(15)
        self.password_lineEdit.setFont(font)
        self.password_lineEdit.setStyleSheet("background-color: rgb(255, 255, 255);\n""border:none;\n""border-bottom: 3px solid rgba(46,82,101,200);\n""color:rgba(0,0,0,240);\n""padding-bottom:7px;")
        self.password_lineEdit.setEchoMode(QtWidgets.QLineEdit.Password)
        self.password_lineEdit.setObjectName("password_lineEdit")

        self.login_pushButton = QtWidgets.QPushButton(self.widget)
        self.login_pushButton.setGeometry(QtCore.QRect(480, 480, 181, 51))
        font = QtGui.QFont()
        font.setPointSize(15)
        font.setBold(True)
        font.setWeight(75)
        self.login_pushButton.setFont(font)
        self.login_pushButton.setStyleSheet("background-color: rgb(153, 105, 199);\n""border-radius:6px;\n""")
        self.login_pushButton.setObjectName("login_pushButton")
        self.login_pushButton.clicked.connect(self.openSearchwindow)
        
        self.caution_label = QtWidgets.QLabel(self.widget)
        self.caution_label.setGeometry(QtCore.QRect(460, 550, 271, 31))
        self.caution_label.setStyleSheet("background-color: rgb(255, 255, 255);")
        self.caution_label.setText("")
        self.caution_label.setObjectName("caution_label")

        self.label_2.raise_()
        self.img_label.raise_()
        self.label_3.raise_()
        self.username_lineEdit.raise_()
        self.password_lineEdit.raise_()
        self.login_pushButton.raise_()
        self.caution_label.raise_()
        self.gridLayout.addWidget(self.widget, 0, 1, 1, 1)

        self.retranslateUi(Form)
        QtCore.QMetaObject.connectSlotsByName(Form)

    def retranslateUi(self, Form):
        _translate = QtCore.QCoreApplication.translate
        Form.setWindowTitle(_translate("Form", "Form"))
        self.label_3.setText(_translate("Form", "Log In"))
        self.username_lineEdit.setPlaceholderText(_translate("Form", "User Name"))
        self.password_lineEdit.setPlaceholderText(_translate("Form", "Password"))
        self.login_pushButton.setText(_translate("Form", "Log In"))


if __name__ == "__main__":
    import sys
    app = QtWidgets.QApplication(sys.argv)
    Form = QtWidgets.QWidget()
    ui = Ui_login_Form()
    ui.setupUi(Form)
    Form.show()
    app.exec_()
    del app




## Search Window

In [13]:
from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtWidgets import QFileDialog,QWidget
from PyQt5.QtGui import QPixmap

class Ui_SearchWindow(object):
            
    def setupUi(self, SearchWindow):
        SearchWindow.setObjectName("SearchWindow")
        SearchWindow.resize(865, 711)
        self.centralwidget = QtWidgets.QWidget(SearchWindow)
        self.centralwidget.setObjectName("centralwidget")

        self.img_label = QtWidgets.QLabel(self.centralwidget)
        self.img_label.setGeometry(QtCore.QRect(10, 10, 431, 661))
        self.img_label.setStyleSheet("border-image: url(shopping.png);\n""border-top-left-radius :50px;")
        self.img_label.setText("")
        self.img_label.setObjectName("img_label")

        self.label_2 = QtWidgets.QLabel(self.centralwidget)
        self.label_2.setGeometry(QtCore.QRect(440, 10, 401, 651))
        self.label_2.setStyleSheet("background-color: rgb(255, 255, 255);\n""border-bottom-right-radius :50px;")
        self.label_2.setText("")
        self.label_2.setObjectName("label_2")

        self.load_pushButton = QtWidgets.QPushButton(self.centralwidget)
        self.load_pushButton.setGeometry(QtCore.QRect(510, 410, 251, 71))
        font = QtGui.QFont()
        font.setPointSize(15)
        font.setBold(True)
        font.setWeight(75)
        self.load_pushButton.setFont(font)
        self.load_pushButton.setStyleSheet("background-color: rgb(153, 105, 199);\n""border-radius:6px;\n""")
        self.load_pushButton.setObjectName("load_pushButton")
        self.load_pushButton.clicked.connect(self.loadimage)

        self.search_pushButton = QtWidgets.QPushButton(self.centralwidget)
        self.search_pushButton.setGeometry(QtCore.QRect(510, 520, 251, 71))
        font = QtGui.QFont()
        font.setPointSize(15)
        font.setBold(True)
        font.setWeight(75)
        self.search_pushButton.setFont(font)
        self.search_pushButton.setStyleSheet("background-color: rgb(153, 105, 199);\n""border-radius:6px;\n""")
        self.search_pushButton.setObjectName("search_pushButton")
        self.search_pushButton.clicked.connect(self.loadwindow)

        self.product_display_label = QtWidgets.QLabel(self.centralwidget)
        self.product_display_label.setGeometry(QtCore.QRect(480, 50, 321, 311))
        sizePolicy = QtWidgets.QSizePolicy(QtWidgets.QSizePolicy.Ignored, QtWidgets.QSizePolicy.Ignored)
        sizePolicy.setHorizontalStretch(0)
        sizePolicy.setVerticalStretch(0)
        sizePolicy.setHeightForWidth(self.product_display_label.sizePolicy().hasHeightForWidth())
        self.product_display_label.setSizePolicy(sizePolicy)
        self.product_display_label.setText("")
        self.product_display_label.setScaledContents(True)
        self.product_display_label.setObjectName("product_display_label")

        SearchWindow.setCentralWidget(self.centralwidget)
        self.menubar = QtWidgets.QMenuBar(SearchWindow)
        self.menubar.setGeometry(QtCore.QRect(0, 0, 865, 26))
        self.menubar.setObjectName("menubar")
        SearchWindow.setMenuBar(self.menubar)
        self.statusbar = QtWidgets.QStatusBar(SearchWindow)
        self.statusbar.setObjectName("statusbar")
        SearchWindow.setStatusBar(self.statusbar)

        self.retranslateUi(SearchWindow)
        QtCore.QMetaObject.connectSlotsByName(SearchWindow)
        
        
    def loadimage(self):
        file_name = QtWidgets.QFileDialog.getOpenFileName(None, 'Open file', 'F:\\',"Image files (*.jpg *.png *.jpeg)")
        self.pixmap = QPixmap(file_name[0])
        self.product_display_label.setPixmap(self.pixmap)
        f = open("query_imagepath.txt", "w")
        f.write(str(file_name[0]))
        f.close()
        
        with open('query_imagepath.txt') as f:
            contents = f.read()
            
        new_image, x = load_image(str(contents))
        new_features = feat_extractor.predict(x)

        # project it into pca space
        new_pca_features = pca.transform(new_features)[0]

        # calculate its distance to all the other images pca feature vectors
        distances = [ distance.cosine(new_pca_features, feat) for feat in pca_features ]
        idx_closest = sorted(range(len(distances)), key=lambda k: distances[k])[0:10]  
        results_image,paths_final = get_concatenated_images(idx_closest, 200)
        
        
        
        textfile = open("temp.txt","w")
        for ele in paths_final:
            textfile.write(ele+"\n")
        textfile.close()    
        
        
    
    def loadwindow(self):
        self.window = QtWidgets.QMainWindow()
        self.ui = Ui_loadWindow()
        self.ui.setupUi(self.window)
        self.window.show()
        

        
    def retranslateUi(self, SearchWindow):
        _translate = QtCore.QCoreApplication.translate
        SearchWindow.setWindowTitle(_translate("SearchWindow", "SearchWindow"))
        self.load_pushButton.setText(_translate("SearchWindow", "Load Product"))
        self.search_pushButton.setText(_translate("SearchWindow", "Search"))

if __name__ == "__main__":
    import sys
    app = QtWidgets.QApplication(sys.argv)
    SearchWindow = QtWidgets.QMainWindow()
    ui = Ui_SearchWindow()
    ui.setupUi(SearchWindow)
    SearchWindow.show()
    app.exec_()
    del app



## Load Window

In [14]:
from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *
counter = 0
class Ui_loadWindow(object):
    
    def setupUi(self, loadWindow):
        
        loadWindow.setObjectName("loadWindow")
        loadWindow.resize(818, 289)
        loadWindow.setStyleSheet("background-color: rgb(95, 114, 190);")
        self.centralwidget = QtWidgets.QWidget(loadWindow)
        self.centralwidget.setObjectName("centralwidget")
        self.label = QtWidgets.QLabel(self.centralwidget)
        self.label.setGeometry(QtCore.QRect(310, 10, 261, 101))
        font = QtGui.QFont()
        font.setFamily("Segoe UI Black")
        font.setPointSize(25)
        font.setBold(True)
        font.setWeight(75)

        self.label.setFont(font)
        self.label.setStyleSheet("color: rgb(254, 121, 199);")
        self.label.setObjectName("label")

        self.timer = QtCore.QTimer()
        self.timer.timeout.connect(self.progress)
        self.timer.start(35)

        self.progressBar = QtWidgets.QProgressBar(self.centralwidget)
        self.progressBar.setGeometry(QtCore.QRect(50, 150, 741, 41))
        self.progressBar.setStyleSheet("QProgressBar {\n""\n""    background-color: rgb(98,114,164);\n""    border-style : none;\n""    border-radius : 10px;\n""   colour: rgb(200,200,200)\n""\n""}\n""QProgressBar::chunk{\n""\n""    background-color: qlineargradient(spread:pad, x1:0, y1:0, x2:1, y2:0, stop:0  #5f72be, stop:1 #9921e8);\n""    border-radius: 10px;\n""}")
        self.progressBar.setProperty("value", 24)
        self.progressBar.setObjectName("progressBar")



        loadWindow.setCentralWidget(self.centralwidget)
        self.statusbar = QtWidgets.QStatusBar(loadWindow)
        self.statusbar.setObjectName("statusbar")
        loadWindow.setStatusBar(self.statusbar)

        self.retranslateUi(loadWindow)
        QtCore.QMetaObject.connectSlotsByName(loadWindow)

    def progress(self):
        global counter
        self.progressBar.setValue(counter)
        if counter > 100:
            self.timer.stop()
            self.window = QtWidgets.QMainWindow()
            self.ui = Ui_ProductWindow()
            self.ui.setupUi(self.window)
            self.window.show()
            loadWindow.close()
            
            
        counter += 1
        

    def retranslateUi(self, loadWindow):
        _translate = QtCore.QCoreApplication.translate
        loadWindow.setWindowTitle(_translate("loadWindow", "loadWindow"))
        self.label.setText(_translate("loadWindow", "Loading.."))


if __name__ == "__main__":
    import sys
    app = QtWidgets.QApplication(sys.argv)
    loadWindow = QtWidgets.QMainWindow()
    ui = Ui_loadWindow()
    ui.setupUi(loadWindow)
    loadWindow.show()
    app.exec_()
    del app



## Product Display Window

In [15]:
import pandas as pd
fileName = 'Images Dataset Product Description.xlsx'
data_only = pd.read_excel(fileName, index_col=False)
data = pd.DataFrame(data_only)
data.drop(data.columns[[0,2]], axis=1, inplace=True)

In [16]:
def getLabelData(queryIMAGE):
    fileName = 'Images Dataset Product Description.xlsx'
    data_only = pd.read_excel(fileName, index_col=False)
    data = pd.DataFrame(data_only)
    data.drop(data.columns[[0,2]], axis=1, inplace=True)
    
    columns = ['PATH : ','PRODUCT : ', 'COLOR : ', 'TYPE : ', 'BRAND : ', 'RATING : ', 'PRICE : ']
    datalist = data.values.tolist()
    ansList = []
    outString = ''
    for rowList in datalist:
        if queryIMAGE in rowList:
            ansList = rowList
              
    return ansList



In [17]:
from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtWidgets import * 
from PyQt5.QtGui import * 
from PyQt5.QtCore import *

class Ui_ProductWindow(QtWidgets.QWidget):
    
    global items_list
    items_list = []
    def opendialog(self):
        sender_button = self.sender()  
        product_index = str(sender_button.objectName()).split('_')[1]
        quanities = ['1','2','3','4','5']
        quantity, response = QtWidgets.QInputDialog.getItem(self, 'Add to cart', 'Quantity',quanities)
        if response == False:
            pass
        else:
            items_list.append([product_index,quantity])
        f = open("cartdetails.txt", "w")
        for l in items_list:
            for ele in l:
                f.write(str(ele)+'\t')
            f.write('\n')
        f.close()
    
            
                          
    
    def setupUi(self, ProductWindow):
        ProductWindow.setObjectName("ProductWindow")
        ProductWindow.resize(889, 712)

        self.centralwidget = QtWidgets.QWidget(ProductWindow)
        self.centralwidget.setObjectName("centralwidget")
        self.centralwidget.setStyleSheet("background-color: rgb(181, 137, 214);")
        self.verticalLayout = QtWidgets.QVBoxLayout(self.centralwidget)
        self.verticalLayout.setObjectName("verticalLayout")

        self.scrollArea = QtWidgets.QScrollArea(self.centralwidget)
        self.scrollArea.setWidgetResizable(True)
        self.scrollArea.setObjectName("scrollArea")

        self.scrollAreaWidgetContents = QtWidgets.QWidget()
        self.scrollAreaWidgetContents.setGeometry(QtCore.QRect(0, 0, 844, 2585))
        self.scrollAreaWidgetContents.setObjectName("scrollAreaWidgetContents")

        self.formLayout = QtWidgets.QFormLayout(self.scrollAreaWidgetContents)
        self.formLayout.setObjectName("formLayout")

        self.pushButton_1 = QtWidgets.QPushButton(self.scrollAreaWidgetContents)
        sizePolicy = QtWidgets.QSizePolicy(QtWidgets.QSizePolicy.Fixed, QtWidgets.QSizePolicy.Fixed)
        sizePolicy.setHorizontalStretch(0)
        sizePolicy.setVerticalStretch(0)
        sizePolicy.setHeightForWidth(self.pushButton_1.sizePolicy().hasHeightForWidth())
        self.pushButton_1.setSizePolicy(sizePolicy)
        self.pushButton_1.setMinimumSize(QtCore.QSize(250, 250))
        self.pushButton_1.setObjectName("pushButton_1")
        self.formLayout.setWidget(0, QtWidgets.QFormLayout.LabelRole, self.pushButton_1)   
        
        self.pushButton_2 = QtWidgets.QPushButton(self.scrollAreaWidgetContents)
        sizePolicy = QtWidgets.QSizePolicy(QtWidgets.QSizePolicy.Fixed, QtWidgets.QSizePolicy.Fixed)
        sizePolicy.setHorizontalStretch(0)
        sizePolicy.setVerticalStretch(0)
        sizePolicy.setHeightForWidth(self.pushButton_2.sizePolicy().hasHeightForWidth())
        self.pushButton_2.setSizePolicy(sizePolicy)
        self.pushButton_2.setMinimumSize(QtCore.QSize(250, 250))
        self.pushButton_2.setObjectName("pushButton_2")
        self.formLayout.setWidget(1, QtWidgets.QFormLayout.LabelRole, self.pushButton_2)

        self.pushButton_3 = QtWidgets.QPushButton(self.scrollAreaWidgetContents)
        sizePolicy = QtWidgets.QSizePolicy(QtWidgets.QSizePolicy.Fixed, QtWidgets.QSizePolicy.Fixed)
        sizePolicy.setHorizontalStretch(0)
        sizePolicy.setVerticalStretch(0)
        sizePolicy.setHeightForWidth(self.pushButton_3.sizePolicy().hasHeightForWidth())
        self.pushButton_3.setSizePolicy(sizePolicy)
        self.pushButton_3.setMinimumSize(QtCore.QSize(250, 250))
        self.pushButton_3.setObjectName("pushButton_3")
        self.formLayout.setWidget(2, QtWidgets.QFormLayout.LabelRole, self.pushButton_3)

        self.pushButton_4 = QtWidgets.QPushButton(self.scrollAreaWidgetContents)
        sizePolicy = QtWidgets.QSizePolicy(QtWidgets.QSizePolicy.Minimum, QtWidgets.QSizePolicy.Preferred)
        sizePolicy.setHorizontalStretch(0)
        sizePolicy.setVerticalStretch(0)
        sizePolicy.setHeightForWidth(self.pushButton_4.sizePolicy().hasHeightForWidth())
        self.pushButton_4.setSizePolicy(sizePolicy)
        self.pushButton_4.setMinimumSize(QtCore.QSize(250, 250))
        self.pushButton_4.setObjectName("pushButton_4")
        self.formLayout.setWidget(3, QtWidgets.QFormLayout.LabelRole, self.pushButton_4)

        self.pushButton_5 = QtWidgets.QPushButton(self.scrollAreaWidgetContents)
        sizePolicy = QtWidgets.QSizePolicy(QtWidgets.QSizePolicy.Minimum, QtWidgets.QSizePolicy.Preferred)
        sizePolicy.setHorizontalStretch(0)
        sizePolicy.setVerticalStretch(0)
        sizePolicy.setHeightForWidth(self.pushButton_5.sizePolicy().hasHeightForWidth())
        self.pushButton_5.setSizePolicy(sizePolicy)
        self.pushButton_5.setMinimumSize(QtCore.QSize(250, 250))
        self.pushButton_5.setObjectName("pushButton_5")
        self.formLayout.setWidget(4, QtWidgets.QFormLayout.LabelRole, self.pushButton_5)

        self.pushButton_6 = QtWidgets.QPushButton(self.scrollAreaWidgetContents)
        sizePolicy = QtWidgets.QSizePolicy(QtWidgets.QSizePolicy.Minimum, QtWidgets.QSizePolicy.Preferred)
        sizePolicy.setHorizontalStretch(0)
        sizePolicy.setVerticalStretch(0)
        sizePolicy.setHeightForWidth(self.pushButton_6.sizePolicy().hasHeightForWidth())
        self.pushButton_6.setSizePolicy(sizePolicy)
        self.pushButton_6.setMinimumSize(QtCore.QSize(250, 250))
        self.pushButton_6.setObjectName("pushButton_6")
        self.formLayout.setWidget(5, QtWidgets.QFormLayout.LabelRole, self.pushButton_6)

        self.pushButton_7 = QtWidgets.QPushButton(self.scrollAreaWidgetContents)
        sizePolicy = QtWidgets.QSizePolicy(QtWidgets.QSizePolicy.Minimum, QtWidgets.QSizePolicy.Preferred)
        sizePolicy.setHorizontalStretch(0)
        sizePolicy.setVerticalStretch(0)
        sizePolicy.setHeightForWidth(self.pushButton_7.sizePolicy().hasHeightForWidth())
        self.pushButton_7.setSizePolicy(sizePolicy)
        self.pushButton_7.setMinimumSize(QtCore.QSize(250, 250))
        self.pushButton_7.setObjectName("pushButton_7")
        self.formLayout.setWidget(6, QtWidgets.QFormLayout.LabelRole, self.pushButton_7)

        self.pushButton_8 = QtWidgets.QPushButton(self.scrollAreaWidgetContents)
        sizePolicy = QtWidgets.QSizePolicy(QtWidgets.QSizePolicy.Minimum, QtWidgets.QSizePolicy.Preferred)
        sizePolicy.setHorizontalStretch(0)
        sizePolicy.setVerticalStretch(0)
        sizePolicy.setHeightForWidth(self.pushButton_8.sizePolicy().hasHeightForWidth())
        self.pushButton_8.setSizePolicy(sizePolicy)
        self.pushButton_8.setMinimumSize(QtCore.QSize(250, 250))
        self.pushButton_8.setObjectName("pushButton_8")
        self.formLayout.setWidget(7, QtWidgets.QFormLayout.LabelRole, self.pushButton_8)

        self.pushButton_9 = QtWidgets.QPushButton(self.scrollAreaWidgetContents)
        sizePolicy = QtWidgets.QSizePolicy(QtWidgets.QSizePolicy.Minimum, QtWidgets.QSizePolicy.Preferred)
        sizePolicy.setHorizontalStretch(0)
        sizePolicy.setVerticalStretch(0)
        sizePolicy.setHeightForWidth(self.pushButton_9.sizePolicy().hasHeightForWidth())
        self.pushButton_9.setSizePolicy(sizePolicy)
        self.pushButton_9.setMinimumSize(QtCore.QSize(250, 250))
        self.pushButton_9.setObjectName("pushButton_9")
        self.formLayout.setWidget(8, QtWidgets.QFormLayout.LabelRole, self.pushButton_9)

        self.pushButton_10 = QtWidgets.QPushButton(self.scrollAreaWidgetContents)
        sizePolicy = QtWidgets.QSizePolicy(QtWidgets.QSizePolicy.Minimum, QtWidgets.QSizePolicy.Preferred)
        sizePolicy.setHorizontalStretch(0)
        sizePolicy.setVerticalStretch(0)
        sizePolicy.setHeightForWidth(self.pushButton_10.sizePolicy().hasHeightForWidth())
        self.pushButton_10.setSizePolicy(sizePolicy)
        self.pushButton_10.setMinimumSize(QtCore.QSize(250, 250))
        self.pushButton_10.setObjectName("pushButton_10")
        self.formLayout.setWidget(9, QtWidgets.QFormLayout.LabelRole, self.pushButton_10)

            
        finals = []

        with open('temp.txt', 'r') as file:
            for row in file.readlines():
                row = row.split('\\')
                finals.append(row[1][:-1])
                
            
       
        self.pushButton_1.setStyleSheet("border-image:url(dataset/"+finals[0]+")")
        self.pushButton_2.setStyleSheet("border-image:url(dataset/"+finals[1]+")")
        self.pushButton_3.setStyleSheet("border-image:url(dataset/"+finals[2]+")")
        self.pushButton_4.setStyleSheet("border-image:url(dataset/"+finals[3]+")")
        self.pushButton_5.setStyleSheet("border-image:url(dataset/"+finals[4]+")")
        self.pushButton_6.setStyleSheet("border-image:url(dataset/"+finals[5]+")")
        self.pushButton_7.setStyleSheet("border-image:url(dataset/"+finals[6]+")")
        self.pushButton_8.setStyleSheet("border-image:url(dataset/"+finals[7]+")")
        self.pushButton_9.setStyleSheet("border-image:url(dataset/"+finals[8]+")")
        self.pushButton_10.setStyleSheet("border-image:url(dataset/"+finals[9]+")")
        
        
        self.pushButton_1.clicked.connect(self.opendialog)
        self.pushButton_2.clicked.connect(self.opendialog)
        self.pushButton_3.clicked.connect(self.opendialog)
        self.pushButton_4.clicked.connect(self.opendialog)
        self.pushButton_5.clicked.connect(self.opendialog)
        self.pushButton_6.clicked.connect(self.opendialog)
        self.pushButton_7.clicked.connect(self.opendialog)
        self.pushButton_8.clicked.connect(self.opendialog)
        self.pushButton_9.clicked.connect(self.opendialog)
        self.pushButton_10.clicked.connect(self.opendialog)
                    
    
        self.label_1 = QtWidgets.QLabel(self.scrollAreaWidgetContents)
        self.label_1.setObjectName("label_1")
        self.formLayout.setWidget(0, QtWidgets.QFormLayout.FieldRole, self.label_1)
        self.label_1.setStyleSheet("background-color: rgb(255, 255, 255);")

        self.label_2 = QtWidgets.QLabel(self.scrollAreaWidgetContents)
        self.label_2.setObjectName("label_2")
        self.formLayout.setWidget(1, QtWidgets.QFormLayout.FieldRole, self.label_2)
        self.label_2.setStyleSheet("background-color: rgb(255, 255, 255);")

        self.label_3 = QtWidgets.QLabel(self.scrollAreaWidgetContents)
        self.label_3.setObjectName("label_3")
        self.formLayout.setWidget(2, QtWidgets.QFormLayout.FieldRole, self.label_3)
        self.label_3.setStyleSheet("background-color: rgb(255, 255, 255);")

        self.label_4 = QtWidgets.QLabel(self.scrollAreaWidgetContents)
        self.label_4.setObjectName("label_4")
        self.formLayout.setWidget(3, QtWidgets.QFormLayout.FieldRole, self.label_4)
        self.label_4.setStyleSheet("background-color: rgb(255, 255, 255);")

        self.label_5 = QtWidgets.QLabel(self.scrollAreaWidgetContents)
        self.label_5.setObjectName("label_5")
        self.formLayout.setWidget(4, QtWidgets.QFormLayout.FieldRole, self.label_5)
        self.label_5.setStyleSheet("background-color: rgb(255, 255, 255);")

        self.label_6 = QtWidgets.QLabel(self.scrollAreaWidgetContents)
        self.label_6.setObjectName("label_6")
        self.formLayout.setWidget(5, QtWidgets.QFormLayout.FieldRole, self.label_6)
        self.label_6.setStyleSheet("background-color: rgb(255, 255, 255);")

        self.label_7 = QtWidgets.QLabel(self.scrollAreaWidgetContents)
        self.label_7.setObjectName("label_7")
        self.formLayout.setWidget(6, QtWidgets.QFormLayout.FieldRole, self.label_7)
        self.label_7.setStyleSheet("background-color: rgb(255, 255, 255);")

        self.label_8 = QtWidgets.QLabel(self.scrollAreaWidgetContents)
        self.label_8.setObjectName("label_8")
        self.formLayout.setWidget(7, QtWidgets.QFormLayout.FieldRole, self.label_8)
        self.label_8.setStyleSheet("background-color: rgb(255, 255, 255);")

        self.label_9 = QtWidgets.QLabel(self.scrollAreaWidgetContents)
        self.label_9.setObjectName("label_9")
        self.formLayout.setWidget(8, QtWidgets.QFormLayout.FieldRole, self.label_9)
        self.label_9.setStyleSheet("background-color: rgb(255, 255, 255);")

        self.label_10 = QtWidgets.QLabel(self.scrollAreaWidgetContents)
        self.label_10.setObjectName("label_10")
        self.formLayout.setWidget(9, QtWidgets.QFormLayout.FieldRole, self.label_10)
        self.label_10.setStyleSheet("background-color: rgb(255, 255, 255);")
            
        
        

        self.scrollArea.setWidget(self.scrollAreaWidgetContents)
        self.verticalLayout.addWidget(self.scrollArea)

        self.next_pushButton = QtWidgets.QPushButton(self.centralwidget)
        self.next_pushButton.setMinimumSize(QtCore.QSize(0, 50))
        self.next_pushButton.setStyleSheet("background-color: rgb(153, 105, 199);\n""border-radius:6px;\n""")
        self.next_pushButton.setObjectName("next_pushButton")
        self.next_pushButton.clicked.connect(self.viewsummary)
        
        
        self.verticalLayout.addWidget(self.next_pushButton)

        ProductWindow.setCentralWidget(self.centralwidget)
        self.menubar = QtWidgets.QMenuBar(ProductWindow)
        self.menubar.setGeometry(QtCore.QRect(0, 0, 889, 26))
        self.menubar.setObjectName("menubar")
        ProductWindow.setMenuBar(self.menubar)
        self.statusbar = QtWidgets.QStatusBar(ProductWindow)
        self.statusbar.setObjectName("statusbar")
        ProductWindow.setStatusBar(self.statusbar)
        self.retranslateUi(ProductWindow)
        QtCore.QMetaObject.connectSlotsByName(ProductWindow)
        
    
        _translate = QtCore.QCoreApplication.translate

        self.label_1.setText(_translate("ProductWindow","<html><head/><body><p><span style=\" font-size:12pt; font-weight:600;\">PRODUCT : </span><span style=\" font-size:12pt;\">"+ getLabelData(finals[0])[1] +
                                      "</span></p><p><span style=\" font-size:12pt; font-weight:600;\">COLOUR : </span><span style=\" font-size:12pt;\">"+ getLabelData(finals[0])[2] +"</span></p><p><span style=\" font-size:12pt; font-weight:600;\">"
                                      "TYPE : </span><span style=\" font-size:12pt;\">"+ getLabelData(finals[0])[3] +"</span></p><p align=\"justify\"><span style=\" font-size:12pt; font-weight:600;\">BRAND : </span><span style=\" font-size:12pt;\">"
                                      ""+ getLabelData(finals[0])[4] +"</span></p><p><span style=\" font-size:12pt; font-weight:600;\">"
                                      "RATING : </span><span style=\" font-size:12pt;\">"+ str(getLabelData(finals[0])[5]) +"</span>"
                                      "</p><p><span style=\" font-size:12pt; font-weight:600;\">PRICE : "
                                      "</span><span style=\" font-size:12pt;\">"+ str(getLabelData(finals[0])[6]) +"</span></p></body></html>"))
        
        self.label_2.setText(_translate("ProductWindow","<html><head/><body><p><span style=\" font-size:12pt; font-weight:600;\">PRODUCT : </span><span style=\" font-size:12pt;\">"+ getLabelData(finals[1])[1] +
                                      "</span></p><p><span style=\" font-size:12pt; font-weight:600;\">COLOUR : </span><span style=\" font-size:12pt;\">"+ getLabelData(finals[1])[2] +"</span></p><p><span style=\" font-size:12pt; font-weight:600;\">"
                                      "TYPE : </span><span style=\" font-size:12pt;\">"+ getLabelData(finals[1])[3] +"</span></p><p align=\"justify\"><span style=\" font-size:12pt; font-weight:600;\">BRAND : </span><span style=\" font-size:12pt;\">"
                                      ""+ getLabelData(finals[1])[4] +"</span></p><p><span style=\" font-size:12pt; font-weight:600;\">"
                                      "RATING : </span><span style=\" font-size:12pt;\">"+ str(getLabelData(finals[1])[5]) +"</span>"
                                      "</p><p><span style=\" font-size:12pt; font-weight:600;\">PRICE : "
                                      "</span><span style=\" font-size:12pt;\">"+ str(getLabelData(finals[1])[6]) +"</span></p></body></html>"))
        
        self.label_3.setText(_translate("ProductWindow","<html><head/><body><p><span style=\" font-size:12pt; font-weight:600;\">PRODUCT : </span><span style=\" font-size:12pt;\">"+ getLabelData(finals[2])[1] +
                                      "</span></p><p><span style=\" font-size:12pt; font-weight:600;\">COLOUR : </span><span style=\" font-size:12pt;\">"+ getLabelData(finals[2])[2] +"</span></p><p><span style=\" font-size:12pt; font-weight:600;\">"
                                      "TYPE : </span><span style=\" font-size:12pt;\">"+ getLabelData(finals[2])[3] +"</span></p><p align=\"justify\"><span style=\" font-size:12pt; font-weight:600;\">BRAND : </span><span style=\" font-size:12pt;\">"
                                      ""+ getLabelData(finals[2])[4] +"</span></p><p><span style=\" font-size:12pt; font-weight:600;\">"
                                      "RATING : </span><span style=\" font-size:12pt;\">"+ str(getLabelData(finals[2])[5]) +"</span>"
                                      "</p><p><span style=\" font-size:12pt; font-weight:600;\">PRICE : "
                                      "</span><span style=\" font-size:12pt;\">"+ str(getLabelData(finals[2])[6]) +"</span></p></body></html>"))
        
        self.label_4.setText(_translate("ProductWindow","<html><head/><body><p><span style=\" font-size:12pt; font-weight:600;\">PRODUCT : </span><span style=\" font-size:12pt;\">"+ getLabelData(finals[3])[1] +
                                      "</span></p><p><span style=\" font-size:12pt; font-weight:600;\">COLOUR : </span><span style=\" font-size:12pt;\">"+ getLabelData(finals[3])[2] +"</span></p><p><span style=\" font-size:12pt; font-weight:600;\">"
                                      "TYPE : </span><span style=\" font-size:12pt;\">"+ getLabelData(finals[3])[3] +"</span></p><p align=\"justify\"><span style=\" font-size:12pt; font-weight:600;\">BRAND : </span><span style=\" font-size:12pt;\">"
                                      ""+ getLabelData(finals[3])[4] +"</span></p><p><span style=\" font-size:12pt; font-weight:600;\">"
                                      "RATING : </span><span style=\" font-size:12pt;\">"+ str(getLabelData(finals[3])[5]) +"</span>"
                                      "</p><p><span style=\" font-size:12pt; font-weight:600;\">PRICE : "
                                      "</span><span style=\" font-size:12pt;\">"+ str(getLabelData(finals[3])[6]) +"</span></p></body></html>"))
        
        self.label_5.setText(_translate("ProductWindow","<html><head/><body><p><span style=\" font-size:12pt; font-weight:600;\">PRODUCT : </span><span style=\" font-size:12pt;\">"+ getLabelData(finals[4])[1] +
                                      "</span></p><p><span style=\" font-size:12pt; font-weight:600;\">COLOUR : </span><span style=\" font-size:12pt;\">"+ getLabelData(finals[4])[2] +"</span></p><p><span style=\" font-size:12pt; font-weight:600;\">"
                                      "TYPE : </span><span style=\" font-size:12pt;\">"+ getLabelData(finals[4])[3] +"</span></p><p align=\"justify\"><span style=\" font-size:12pt; font-weight:600;\">BRAND : </span><span style=\" font-size:12pt;\">"
                                      ""+ getLabelData(finals[4])[4] +"</span></p><p><span style=\" font-size:12pt; font-weight:600;\">"
                                      "RATING : </span><span style=\" font-size:12pt;\">"+ str(getLabelData(finals[4])[5]) +"</span>"
                                      "</p><p><span style=\" font-size:12pt; font-weight:600;\">PRICE : "
                                      "</span><span style=\" font-size:12pt;\">"+ str(getLabelData(finals[4])[6]) +"</span></p></body></html>"))
        
        self.label_6.setText(_translate("ProductWindow","<html><head/><body><p><span style=\" font-size:12pt; font-weight:600;\">PRODUCT : </span><span style=\" font-size:12pt;\">"+ getLabelData(finals[5])[1] +
                                      "</span></p><p><span style=\" font-size:12pt; font-weight:600;\">COLOUR : </span><span style=\" font-size:12pt;\">"+ getLabelData(finals[5])[2] +"</span></p><p><span style=\" font-size:12pt; font-weight:600;\">"
                                      "TYPE : </span><span style=\" font-size:12pt;\">"+ getLabelData(finals[5])[3] +"</span></p><p align=\"justify\"><span style=\" font-size:12pt; font-weight:600;\">BRAND : </span><span style=\" font-size:12pt;\">"
                                      ""+ getLabelData(finals[5])[4] +"</span></p><p><span style=\" font-size:12pt; font-weight:600;\">"
                                      "RATING : </span><span style=\" font-size:12pt;\">"+ str(getLabelData(finals[5])[5]) +"</span>"
                                      "</p><p><span style=\" font-size:12pt; font-weight:600;\">PRICE : "
                                      "</span><span style=\" font-size:12pt;\">"+ str(getLabelData(finals[5])[6]) +"</span></p></body></html>"))
        
        self.label_7.setText(_translate("ProductWindow","<html><head/><body><p><span style=\" font-size:12pt; font-weight:600;\">PRODUCT : </span><span style=\" font-size:12pt;\">"+ getLabelData(finals[6])[1] +
                                      "</span></p><p><span style=\" font-size:12pt; font-weight:600;\">COLOUR : </span><span style=\" font-size:12pt;\">"+ getLabelData(finals[6])[2] +"</span></p><p><span style=\" font-size:12pt; font-weight:600;\">"
                                      "TYPE : </span><span style=\" font-size:12pt;\">"+ getLabelData(finals[6])[3] +"</span></p><p align=\"justify\"><span style=\" font-size:12pt; font-weight:600;\">BRAND : </span><span style=\" font-size:12pt;\">"
                                      ""+ getLabelData(finals[6])[4] +"</span></p><p><span style=\" font-size:12pt; font-weight:600;\">"
                                      "RATING : </span><span style=\" font-size:12pt;\">"+ str(getLabelData(finals[6])[5]) +"</span>"
                                      "</p><p><span style=\" font-size:12pt; font-weight:600;\">PRICE : "
                                      "</span><span style=\" font-size:12pt;\">"+ str(getLabelData(finals[6])[6]) +"</span></p></body></html>"))
        
        self.label_8.setText(_translate("ProductWindow","<html><head/><body><p><span style=\" font-size:12pt; font-weight:600;\">PRODUCT : </span><span style=\" font-size:12pt;\">"+ getLabelData(finals[7])[1] +
                                      "</span></p><p><span style=\" font-size:12pt; font-weight:600;\">COLOUR : </span><span style=\" font-size:12pt;\">"+ getLabelData(finals[7])[2] +"</span></p><p><span style=\" font-size:12pt; font-weight:600;\">"
                                      "TYPE : </span><span style=\" font-size:12pt;\">"+ getLabelData(finals[7])[3] +"</span></p><p align=\"justify\"><span style=\" font-size:12pt; font-weight:600;\">BRAND : </span><span style=\" font-size:12pt;\">"
                                      ""+ getLabelData(finals[7])[4] +"</span></p><p><span style=\" font-size:12pt; font-weight:600;\">"
                                      "RATING : </span><span style=\" font-size:12pt;\">"+ str(getLabelData(finals[7])[5]) +"</span>"
                                      "</p><p><span style=\" font-size:12pt; font-weight:600;\">PRICE : "
                                      "</span><span style=\" font-size:12pt;\">"+ str(getLabelData(finals[7])[6]) +"</span></p></body></html>"))
        
        self.label_9.setText(_translate("ProductWindow","<html><head/><body><p><span style=\" font-size:12pt; font-weight:600;\">PRODUCT : </span><span style=\" font-size:12pt;\">"+ getLabelData(finals[8])[1] +
                                      "</span></p><p><span style=\" font-size:12pt; font-weight:600;\">COLOUR : </span><span style=\" font-size:12pt;\">"+ getLabelData(finals[8])[2] +"</span></p><p><span style=\" font-size:12pt; font-weight:600;\">"
                                      "TYPE : </span><span style=\" font-size:12pt;\">"+ getLabelData(finals[8])[3] +"</span></p><p align=\"justify\"><span style=\" font-size:12pt; font-weight:600;\">BRAND : </span><span style=\" font-size:12pt;\">"
                                      ""+ getLabelData(finals[8])[4] +"</span></p><p><span style=\" font-size:12pt; font-weight:600;\">"
                                      "RATING : </span><span style=\" font-size:12pt;\">"+ str(getLabelData(finals[8])[5]) +"</span>"
                                      "</p><p><span style=\" font-size:12pt; font-weight:600;\">PRICE : "
                                      "</span><span style=\" font-size:12pt;\">"+ str(getLabelData(finals[8])[6]) +"</span></p></body></html>"))
        
        self.label_10.setText(_translate("ProductWindow","<html><head/><body><p><span style=\" font-size:12pt; font-weight:600;\">PRODUCT : </span><span style=\" font-size:12pt;\">"+ getLabelData(finals[9])[1] +
                                      "</span></p><p><span style=\" font-size:12pt; font-weight:600;\">COLOUR : </span><span style=\" font-size:12pt;\">"+ getLabelData(finals[9])[2] +"</span></p><p><span style=\" font-size:12pt; font-weight:600;\">"
                                      "TYPE : </span><span style=\" font-size:12pt;\">"+ getLabelData(finals[9])[3] +"</span></p><p align=\"justify\"><span style=\" font-size:12pt; font-weight:600;\">BRAND : </span><span style=\" font-size:12pt;\">"
                                      ""+ getLabelData(finals[9])[4] +"</span></p><p><span style=\" font-size:12pt; font-weight:600;\">"
                                      "RATING : </span><span style=\" font-size:12pt;\">"+ str(getLabelData(finals[9])[5]) +"</span>"
                                      "</p><p><span style=\" font-size:12pt; font-weight:600;\">PRICE : "
                                      "</span><span style=\" font-size:12pt;\">"+ str(getLabelData(finals[9])[6]) +"</span></p></body></html>"))
        
    

        
    def viewsummary(self):
        cartWindow.main_window()
        
        

    def retranslateUi(self, ProductWindow):
        _translate = QtCore.QCoreApplication.translate
        ProductWindow.setWindowTitle(_translate("ProductWindow", "ProductWindow"))
        self.pushButton_1.setText(_translate("ProductWindow", ""))
        self.pushButton_2.setText(_translate("ProductWindow", ""))
        self.pushButton_3.setText(_translate("ProductWindow", ""))
        self.pushButton_4.setText(_translate("ProductWindow", ""))
        self.pushButton_5.setText(_translate("ProductWindow", ""))
        self.pushButton_6.setText(_translate("ProductWindow", ""))
        self.pushButton_7.setText(_translate("ProductWindow", ""))
        self.pushButton_8.setText(_translate("ProductWindow", ""))
        self.pushButton_9.setText(_translate("ProductWindow", ""))
        self.pushButton_10.setText(_translate("ProductWindow", ""))
        self.next_pushButton.setText(_translate("ProductWindow", "Proceed to Buy"))


if __name__ == "__main__":
    import sys
    app = QtWidgets.QApplication(sys.argv)
    ProductWindow = QtWidgets.QMainWindow()
    ui = Ui_ProductWindow()
    ui.setupUi(ProductWindow)
    ProductWindow.show()
    app.exec_()
    del app



## Cart

In [18]:
import pandas as pd
global cartDFFinal
def readProductDetails(cartDetailsFile, finalDetailsFile):
    cartdetailsLines = []
    with open(cartDetailsFile, 'r') as file:
        cartdetailsLines =  file.read()
    cartdetailsLineStrings = cartdetailsLines.split('\n')
    cartdetailsProductStrings = []
    for strDetails in cartdetailsLineStrings:
        cartdetailsProductStrings.append(strDetails.split('\t'))
    cartdetailsProductStrings.pop()
    cartdetails = []
    for st in cartdetailsProductStrings:
        cartdetails.append(list(st[0:2]))
    finals = []
    with open(finalDetailsFile, 'r') as file:
        for row in file.readlines():
            row = row.split('\\')
            finals.append(row[1][:-1])
    return cartdetails, finals

def getBillDetails(cartDetailsFile, finalDetailsFile):     
    cartDETAILS, finalsLIST = readProductDetails(cartDetailsFile, finalDetailsFile)
    fileName = 'Images Dataset Product Description.xlsx'
    data_only = pd.read_excel(fileName, index_col=False)
    data = pd.DataFrame(data_only)
    data.drop(data.columns[[0]], axis=1, inplace=True)
    dataLISTS = data.values.tolist()  
    billDetails = []
    for l in cartDETAILS:        
        finalsIndex = int(l[0]) - 1
        queryIMAGE = finalsLIST[finalsIndex]
        ansList = []   
        for rowList in dataLISTS:
            if queryIMAGE in rowList:
                ansList = rowList
        smallList = ansList[2:6]
        smallList.append(ansList[7])
        quantity = int(l[1])
        smallList.append(quantity)
        amt = int(ansList[7].split(' ')[1])
        totalPrice ='₹ '+str(quantity*amt)+' /-'
        smallList.append(totalPrice)
        billDetails.append(smallList)                
    return billDetails

cartFile  = 'cartdetails.txt'
finalFile = 'temp.txt'
bill = getBillDetails(cartFile, finalFile)
cartDF = pd.DataFrame(bill, columns = ['PRODUCT', 'COLOR', 'TYPE', 'BRAND', 'PRICE', 'QUANTITY', 'TOTAL PRICE'])
totalBillQuantityListValues = cartDF['QUANTITY'].to_list()
totalBillQuantity = sum(totalBillQuantityListValues)
totalBillQuantityString = str(totalBillQuantity)
totalBillAmountList = cartDF['TOTAL PRICE'].to_list()
totalBillAmountListValues= [int(a.split(' ')[1]) for a in totalBillAmountList]
totalBillAmount = sum(totalBillAmountListValues)
totalBillAmountString = '₹ '+str(totalBillAmount)+' /-'
withFinalPrice = [' ', ' ', ' ', ' ', 'TOTAL : ', totalBillQuantityString, totalBillAmountString]
bill.append(withFinalPrice)
cartDFFinal = pd.DataFrame(bill, columns = ['PRODUCT', 'COLOR', 'TYPE', 'BRAND', 'PRICE', 'QUANTITY', 'TOTAL PRICE'])





In [22]:
import sys
from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *
    
    
    
class cartWindow(QWidget):
    
    def __init__(self):
        QWidget.__init__(self)

        tablemodel = pandasModel(cartDFFinal)
        tableview = QTableView()
        tableview.setModel(tablemodel)

        layout = QVBoxLayout(self)
        layout.addWidget(tableview)
        self.setLayout(layout)
        
    def main_window(self):
        self.show()   
        
    
        

class pandasModel(QAbstractTableModel):

    def __init__(self, data):
        QAbstractTableModel.__init__(self)
        self._data = data

    def rowCount(self, parent=None):
        return self._data.shape[0]

    def columnCount(self, parnet=None):
        return self._data.shape[1]

    def data(self, index, role=Qt.DisplayRole):
        if index.isValid():
            if role == Qt.DisplayRole:
                return str(self._data.iloc[index.row(), index.column()])
        return None

    def headerData(self, col, orientation, role):
        if orientation == Qt.Horizontal and role == Qt.DisplayRole:
            return self._data.columns[col]
        return None
    
    
    


if __name__ == '__main__':
    app = QtWidgets.QApplication(sys.argv)
    cartWindow = cartWindow()
    cartWindow.resize(1000,500)
    cartWindow.show()
    app.exec_()
    del app
    
    
